In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import smtplib
import ssl
import webbrowser

# Function to find the price from the given URL
def find_price(URL):
    r = requests.get(URL, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.content, "html.parser")
    try:
        if 'amazon' in URL:
            try:
                price_one = soup.find(id="a-price-whole")
                price = price_one
            except:
                price = price_two
                price_two = soup.find(id="a-price-whole")
            return price
        elif 'flipkart' in URL:
            price = soup.find(class_='_30jeq3 _16Jk6d')
            return price
    except:
        return

# Email sending function
def send_email(price):
    if int(price.replace(',', '')) < 52000:  # Convert price to integer and check if less than 52000
        port = 465  # For SSL
        smtp_server = "smtp.gmail.com"
        sender_email = "pranavpanchal300030@gmail.com"  # Your email address
        password = getpass.getpass("Enter Password:")  # Your email password
        receiver_email = "panipanchal25@gmail.com"  # Recipient email address

        message = f"""\
        Subject: Price Drop Alert

        The price of the product has dropped to {price}.
        Check the link: {URL}
        """

        context = ssl.create_default_context()
        with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message)

i = 1
URL = input("Enter the URL: ")

# Initial price
previous_price = None

while True:
    price_element = find_price(URL)
    
    if price_element is None:
        print("Invalid link")
        break
    
    current_price = price_element.get_text()
    
    print(f"Your current price is: {current_price}")
    
    if i > 2 and current_price != previous_price:
        send_email(current_price)
        webbrowser.open_new(URL)
    
    previous_price = current_price
    i += 1
    
    time.sleep(60)